# Скачивание/импорт пакетов

In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import re
from datasets import load_dataset
from collections import Counter
import nltk
from nltk.corpus import stopwords

/opt/homebrew/Caskroom/miniforge/base/envs/env_tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Методы для пред обработке данных, работе с ними

In [4]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fanfurick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\n', ' ', text)
    return text

In [6]:
def process_dataset_in_chunks(file_path, column_name, chunksize=1000):
    combined_texts = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk[column_name] = chunk[column_name].apply(clean_text)
        combined_texts.append(chunk[column_name])
    return pd.concat(combined_texts, ignore_index=True)

In [7]:
def get_word_counts(text_series):
    all_text = ' '.join(text_series)
    words = all_text.split()
    words = [word for word in words if word not in stop_words]
    word_counts = Counter(words)
    return word_counts

In [8]:
def get_top_words(word_counts, top_n=10):
    most_common = word_counts.most_common(top_n)
    least_common = word_counts.most_common()[:-top_n-1:-1]
    return most_common, least_common

# Датасеты

In [9]:
splits = {'train': 'train.csv', 'validation': 'valid.csv'}
df1_texts = process_dataset_in_chunks("hf://datasets/MonoHime/ru_sentiment_dataset/" + splits["train"], 'text')

'HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/MonoHime/ru_sentiment_dataset/resolve/main/train.csv
Retrying in 1s [Retry 1/5].


In [10]:
df2_texts = process_dataset_in_chunks("hf://datasets/Romjiik/Russian_bank_reviews/final_review_dataset_extended.csv", 'review')

# Различные ответы по датасетам

In [11]:
df1_word_counts = get_word_counts(df1_texts)
df1_most_common, df1_least_common = get_top_words(df1_word_counts)
print("Первый датасет - Самые частые слова:", df1_most_common)
print("Первый датасет - Самые редкие слова:", df1_least_common)

Первый датасет - Самые частые слова: [('очень', 139823), ('это', 128531), ('отель', 46809), ('спасибо', 45783), ('просто', 39224), ('года', 35249), ('время', 29662), ('день', 28402), ('нам', 28198), ('номер', 27202)]
Первый датасет - Самые редкие слова: [('шутливые', 1), ('сготовленные', 1), ('автобарселона', 1), ('близкобрали', 1), ('путешествиясотрудники', 1), ('шрейбер', 1), ('хотнелось', 1), ('любезныймы', 1), ('постернак', 1), ('хруцкой', 1)]


In [12]:
df2_word_counts = get_word_counts(df2_texts)
df2_most_common, df2_least_common = get_top_words(df2_word_counts)
print("Второй датасет - Самые частые слова:", df2_most_common)
print("Второй датасет - Самые редкие слова:", df2_least_common)

Второй датасет - Самые частые слова: [('банка', 15223), ('банк', 11778), ('это', 8475), ('карту', 8128), ('очень', 5625), ('карты', 5540), ('деньги', 5462), ('день', 4443), ('дней', 3828), ('время', 3231)]
Второй датасет - Самые редкие слова: [('вову', 1), ('сотрудникпредставитель', 1), ('делили', 1), ('нба', 1), ('представитедем', 1), ('водный', 1), ('возрастеили', 1), ('пугая', 1), ('схеменепонятно', 1), ('исков', 1)]


In [13]:
combined_texts = pd.concat([df1_texts, df2_texts], ignore_index=True)
combined_texts.to_csv('combined_texts.csv', index=False, header=['text'], encoding='utf-8')
combined_word_counts = get_word_counts(combined_texts)
word_counts_df = pd.DataFrame(combined_word_counts.items(), columns=['word', 'count'])

In [14]:
combined_most_common, combined_least_common = get_top_words(combined_word_counts)
print("Объединённый датасет - Самые частые слова:", combined_most_common)
print("Объединённый датасет - Самые редкие слова:", combined_least_common)

Объединённый датасет - Самые частые слова: [('очень', 145448), ('это', 137006), ('спасибо', 48230), ('отель', 46812), ('просто', 41824), ('года', 38064), ('время', 32893), ('день', 32845), ('нам', 28915), ('номер', 28639)]
Объединённый датасет - Самые редкие слова: [('сотрудникпредставитель', 1), ('нба', 1), ('представитедем', 1), ('возрастеили', 1), ('схеменепонятно', 1), ('выпросьба', 1), ('чатеще', 1), ('ожидатьно', 1), ('средствамимы', 1), ('выполнитьтак', 1)]
